# Présentation

Ce code permet de générer le `csv` de soumission à partir du modèle de regression

# Importation

In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import pad_sequences

import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Notebook
from IPython.display import clear_output
clear_output()

# Importation des données

In [2]:
dateparse = lambda x: datetime.strptime(x, '%d-%m-%Y')
df = pd.read_csv("Challenge_2_submission_template.csv", parse_dates=['Date of Harvest'], date_parser=dateparse)

borne = [
    (-0.0440750, 1.1379025),
    (-0.0215250, 1.1155175),
    (-0.1473925, 1.13914),
    (0.04604250, 1.2140225),
    (21762.0000, 56916.0),
    (293.0, 48423.0)
]
mini = np.array([
    -0.0440750,
    -0.0215250,
    -0.1473925,
    0.04604250,
    21762.0000,
    293.0,
])
multi = np.array([
    1.1819775, 
    1.1370425, 
    1.2865325, 
    1.16798, 
    35154.0, 
    48130.0
])

x = []
for i in range(len(df)):
    with open(f"inputs_lstm_val/{i}.npy", "rb") as f:
        arr = np.load(f)
        arr = np.transpose(arr, (0, 2, 3, 1))
        arr = (arr-mini)/multi
        x.append(arr)

x = pad_sequences(x, dtype="float", maxlen=32)

# Prédiction

## Importation du modèle entraîné

In [3]:
model = load_model("models_finaux/regression/102-0.02253.h5")

clear_output()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 98, 98, 3)         984       
                                                                 
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 2, 2048)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 32)                262176    
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                        

In [4]:
y = model.predict(x)

2023-05-11 11:19:51.979118: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 768000000 exceeds 10% of free system memory.
2023-05-11 11:19:53.342000: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 245760000 exceeds 10% of free system memory.
2023-05-11 11:19:53.342030: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 245760000 exceeds 10% of free system memory.
2023-05-11 11:19:53.342055: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 245760000 exceeds 10% of free system memory.
2023-05-11 11:19:53.701519: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 245760000 exceeds 10% of free system memory.


4/4 [==============================] - 13s 3s/step


In [5]:
df["Predicted Rice Yield (kg/ha)"] = y * (8000-5200) + 5200

In [6]:
df.to_csv("5-Regression.csv")